### DATASET

In [1]:
!git clone https://github.com/UniversalDependencies/UD_Chinese-GSDSimp.git

fatal: destination path 'UD_Chinese-GSDSimp' already exists and is not an empty directory.


In [2]:
!pip install hmmlearn

import numpy as np
from hmmlearn import hmm
from collections import defaultdict

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


### TRAINING HMM

In [3]:
from math import log

# Step 1: Load and preprocess the CoNLL formatted data
def read_conllu_data(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        sentence = []
        pos_tags = []
        for line in f:
            if line.strip() == "":
                if sentence and pos_tags:
                    data.append((sentence, pos_tags))
                    sentence = []
                    pos_tags = []
            else:
                parts = line.strip().split("\t")
                if len(parts) > 1:
                    sentence.append(parts[1])  # Word
                    pos_tags.append(parts[3])   # POS tag
    return data

# Step 2: HMM Training - Calculate transition and emission probabilities with Laplace Smoothing
def train_hmm(data, laplace_smoothing=0.01):
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    pos_counts = defaultdict(int)
    
    for sentence, pos_tags in data:
        prev_pos = "<START>"
        for i in range(len(sentence)):
            word, pos = sentence[i], pos_tags[i]
            transition_counts[prev_pos][pos] += 1
            emission_counts[pos][word] += 1
            pos_counts[pos] += 1
            prev_pos = pos
        # Mark the end of the sentence
        transition_counts[prev_pos]["<END>"] += 1
    
    # Calculate transition probabilities with Laplace smoothing
    transition_probs = {}
    for prev_pos, next_pos_dict in transition_counts.items():
        total_count = sum(next_pos_dict.values()) + laplace_smoothing * len(pos_counts)
        transition_probs[prev_pos] = {pos: (count + laplace_smoothing) / total_count
                                      for pos, count in next_pos_dict.items()}
    
    # Calculate emission probabilities with Laplace smoothing
    emission_probs = {}
    for pos, word_dict in emission_counts.items():
        total_count = sum(word_dict.values()) + laplace_smoothing * (len(emission_counts[pos]) + 1)  # +1 for unseen words
        emission_probs[pos] = {word: (count + laplace_smoothing) / total_count
                               for word, count in word_dict.items()}
    
    return transition_probs, emission_probs, pos_counts

# Log version of handle_unknown_word
def handle_unknown_word(word, emission_probs, pos_counts, smoothing_factor=1e-6):
    if word in emission_probs:
        return log(emission_probs[word])
    elif word.endswith("ing"):
        return log(emission_probs.get("VERB", smoothing_factor))
    elif word[0].isupper():
        return log(emission_probs.get("PROPN", smoothing_factor))
    else:
        return log(emission_probs.get("NOUN", smoothing_factor))


# Update viterbi algorithm to work with log probabilities
def viterbi(sentence, transition_probs, emission_probs, pos_counts):
    pos_tags = list(pos_counts.keys())

    # Initialize Viterbi matrix and backpointer matrix
    viterbi_matrix = np.full((len(pos_tags), len(sentence)), -np.inf)  # log(0) = -inf
    backpointer = np.zeros((len(pos_tags), len(sentence)), dtype=int)

    # Initialization step
    for i, pos in enumerate(pos_tags):
        emission_prob = handle_unknown_word(sentence[0], emission_probs[pos], pos_counts)
        viterbi_matrix[i, 0] = log(transition_probs["<START>"].get(pos, 1e-6)) + emission_prob

    # Recursion step
    for t in range(1, len(sentence)):
        for i, pos in enumerate(pos_tags):
            max_prob = -np.inf
            max_state = 0
            for j, prev_pos in enumerate(pos_tags):
                prob = (viterbi_matrix[j, t-1] + 
                        log(transition_probs[prev_pos].get(pos, 1e-6)) + 
                        handle_unknown_word(sentence[t], emission_probs[pos], pos_counts))
                if prob > max_prob:
                    max_prob = prob
                    max_state = j
            viterbi_matrix[i, t] = max_prob
            backpointer[i, t] = max_state

    # Termination step
    best_last_state = np.argmax(viterbi_matrix[:, len(sentence)-1])

    # Backtrack to find the best path
    best_path = [best_last_state]
    for t in range(len(sentence)-1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, best_last_state)

    # Convert state indices back to POS tags
    best_pos_sequence = [pos_tags[state] for state in best_path]
    return best_pos_sequence

# Step 4: Calculate accuracy
def calculate_accuracy(predicted_tags, true_tags):
    correct = sum(p == t for p, t in zip(predicted_tags, true_tags))
    return correct / len(true_tags) if true_tags else 0.0

# Load your dataset
train_data_path = "UD_Chinese-GSDSimp/zh_gsdsimp-ud-train.conllu"
test_data_path = "UD_Chinese-GSDSimp/zh_gsdsimp-ud-test.conllu"

train_data = read_conllu_data(train_data_path)
test_data = read_conllu_data(test_data_path)

# Train the HMM model with Laplace smoothing (adjust the value of laplace_smoothing)
transition_probs, emission_probs, pos_counts = train_hmm(train_data, laplace_smoothing=0.01)


# Test the HMM model and calculate accuracy
all_predicted_tags = []
all_true_tags = []

for sentence, true_pos_tags in test_data:
    predicted_tags = viterbi(sentence, transition_probs, emission_probs, pos_counts)
    all_predicted_tags.extend(predicted_tags)
    all_true_tags.extend(true_pos_tags)

accuracy = calculate_accuracy(all_predicted_tags, all_true_tags)

# Output accuracy
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.8354


In [6]:
print(len(train_data))
print(len(test_data))

3997
500


### LOGISTIC REGRESSION

In [200]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

# Step 1: Feature extraction
def extract_features_and_labels(data):
    features = []
    labels = []
    
    for sentence, pos_tags in data:
        for i, word in enumerate(sentence):
            features.append({
                'word': word,
                'prev_word': sentence[i - 1] if i > 0 else "<START>",
                'next_word': sentence[i + 1] if i < len(sentence) - 1 else "<END>",
            })
            labels.append(pos_tags[i])
    
    return features, labels

# Extract features and labels from training data
train_features, train_labels = extract_features_and_labels(train_data)

# Convert features to a sparse matrix suitable for Logistic Regression
vectorizer = DictVectorizer(sparse=True)  # Change to sparse=True
X_train = vectorizer.fit_transform(train_features)
y_train = train_labels

# Step 2: Train the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Step 3: Test the Logistic Regression model
test_features, test_labels = extract_features_and_labels(test_data)
X_test = vectorizer.transform(test_features)

# Make predictions
predicted_tags_lr = lr_model.predict(X_test)

# Step 4: Calculate accuracy for Logistic Regression
accuracy_lr = accuracy_score(test_labels, predicted_tags_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr:.4f}")


Logistic Regression Accuracy: 0.8645


### SUPPORT VECTOR MACHINE

In [95]:
from sklearn.svm import SVC
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

# Step 1: Extract features and labels from training data (as defined previously)
train_features, train_labels = extract_features_and_labels(train_data)

# Convert features to a sparse matrix suitable for SVM
vectorizer = DictVectorizer(sparse=True)  # Keep it sparse
X_train = vectorizer.fit_transform(train_features)
y_train = train_labels

# Step 2: Train the Support Vector Machine model
svm_model = SVC(kernel='linear', max_iter=1000)  # You can choose different kernels as needed
svm_model.fit(X_train, y_train)

# Step 3: Test the SVM model
test_features, test_labels = extract_features_and_labels(test_data)
X_test = vectorizer.transform(test_features)

# Make predictions
predicted_tags_svm = svm_model.predict(X_test)

# Step 4: Calculate accuracy for SVM
accuracy_svm = accuracy_score(test_labels, predicted_tags_svm)
print(f"SVM Accuracy: {accuracy_svm:.4f}")


C:\Users\Veda Chatiyode\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 0.7557


### RANDOM FOREST

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

# Step 1: Extract features and labels from training data (as defined previously)
train_features, train_labels = extract_features_and_labels(train_data)

# Convert features to a sparse matrix suitable for Random Forest
vectorizer = DictVectorizer(sparse=True)  # Keep it sparse
X_train = vectorizer.fit_transform(train_features)
y_train = train_labels

# Step 2: Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators as needed
rf_model.fit(X_train, y_train)

# Step 3: Test the Random Forest model
test_features, test_labels = extract_features_and_labels(test_data)
X_test = vectorizer.transform(test_features)

# Make predictions
predicted_tags_rf = rf_model.predict(X_test)

# Step 4: Calculate accuracy for Random Forest
accuracy_rf = accuracy_score(test_labels, predicted_tags_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

Random Forest Accuracy: 0.8117


### LIGHTGBM

In [142]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
import numpy as np  # Import numpy

# Step 1: Prepare the feature matrix and target labels using sparse representation
vectorizer = DictVectorizer(sparse=True)
X = vectorizer.fit_transform(train_features)  # Keep the matrix sparse

# Convert labels to a numpy array to ensure correct format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_labels)  # Encode labels to numeric values

# Step 2: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Prepare LightGBM datasets
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)  # Use free_raw_data=False to keep sparse format
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, free_raw_data=False)

# Step 4: Define LightGBM parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',  # Use 'multiclass' for multi-class classification
    'metric': 'multi_logloss',  # Use 'multi_logloss' for multi-class
    'num_class': len(np.unique(y)),  # Set the number of classes
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

# Step 5: Train the LightGBM model with early stopping using a callback
print("Training LightGBM model with sparse dataset...")
model = lgb.train(params,
                  lgb_train,
                  num_boost_round=100,
                  valid_sets=[lgb_eval],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])

# Step 6: Make predictions on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class for each sample

# Step 7: Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"LightGBM Model Accuracy: {accuracy:.4f}")


Training LightGBM model with sparse dataset...
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.784922
LightGBM Model Accuracy: 0.7366


### LSTM

In [156]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Step 1: Feature extraction
def extract_sentences_and_labels(data):
    sentences = [sentence for sentence, _ in data]
    labels = [pos_tags for _, pos_tags in data]
    return sentences, labels

# Extract sentences and labels from the training and testing data
train_sentences, train_labels = extract_sentences_and_labels(train_data)
test_sentences, test_labels = extract_sentences_and_labels(test_data)

# Step 2: Prepare word and tag dictionaries
all_words = [word for sentence in train_sentences for word in sentence]
all_tags = [tag for tags in train_labels for tag in tags]

# Create a word index and tag index
word2idx = {word: idx + 1 for idx, word in enumerate(set(all_words))}  # +1 because 0 is reserved for padding
tag2idx = {tag: idx for idx, tag in enumerate(set(all_tags))}

# Hyperparameters
MAX_LEN = 50
EMBEDDING_DIM = 64
LSTM_UNITS = 64
NUM_CLASSES = len(tag2idx)

# Step 3: Convert sentences and labels to sequences of indices
def encode_sentences_and_labels(sentences, labels, word2idx, tag2idx, max_len):
    X = [[word2idx.get(word, 0) for word in sentence] for sentence in sentences]  
    y = [[tag2idx[tag] for tag in tags] for tags in labels]
    X_padded = pad_sequences(X, maxlen=max_len, padding='post')
    y_padded = pad_sequences(y, maxlen=max_len, padding='post')
    return X_padded, y_padded

# Encode and pad the training and testing data
X_train, y_train = encode_sentences_and_labels(train_sentences, train_labels, word2idx, tag2idx, MAX_LEN)
X_test, y_test = encode_sentences_and_labels(test_sentences, test_labels, word2idx, tag2idx, MAX_LEN)

# Step 4: Convert labels to categorical values
y_train = [to_categorical(i, num_classes=NUM_CLASSES) for i in y_train]
y_test = [to_categorical(i, num_classes=NUM_CLASSES) for i in y_test]

# Convert y_train and y_test to numpy arrays for LSTM training
y_train = np.array(y_train)
y_test = np.array(y_test)

# Step 5: Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(word2idx) + 1, output_dim=EMBEDDING_DIM, mask_zero=True),  
    Bidirectional(LSTM(units=LSTM_UNITS, return_sequences=True, recurrent_dropout=0.1)),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')  # Output layer for each time step
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), verbose=1)

# Step 7: Evaluate the model
evaluation = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {evaluation[1]:.4f}")

# Step 8: Make predictions on the test set
predicted_tags_lstm = model.predict(X_test)
predicted_tags_lstm = np.argmax(predicted_tags_lstm, axis=-1)

# Step 9: Convert indices back to tag labels
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
predicted_tags = [[idx2tag[idx] for idx in sentence] for sentence in predicted_tags_lstm]
true_tags = [[idx2tag[np.argmax(tag)] for tag in sentence] for sentence in y_test]

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.2483 - loss: 2.3669 - val_accuracy: 0.7478 - val_loss: 1.4590
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.7473 - loss: 1.2857 - val_accuracy: 0.3609 - val_loss: 0.5561
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.3952 - loss: 0.5773 - val_accuracy: 0.3839 - val_loss: 0.3613
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.4290 - loss: 0.3380 - val_accuracy: 0.3899 - val_loss: 0.3081
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.4454 - loss: 0.2464 - val_accuracy: 0.3924 - val_loss: 0.2876
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.4556 - loss: 0.2051 - val_accuracy: 0.3952 - val_loss: 0.2764
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.4600 - loss: 0.1708 - val_accuracy: 0.3958 - val_loss: 0.2779
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.4573 - loss: 0.1487 - val_acc

### N GRAM MODEL

In [114]:
from collections import defaultdict
import random

# Step 1: Preprocess the dataset to extract sentences and POS tags
def read_conllu_data(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        sentence = []
        pos_tags = []
        for line in f:
            if line.strip() == "":
                if sentence and pos_tags:
                    data.append((sentence, pos_tags))
                    sentence = []
                    pos_tags = []
            else:
                parts = line.strip().split("\t")
                if len(parts) > 1:
                    sentence.append(parts[1])  # Word
                    pos_tags.append(parts[3])   # POS tag
    return data

# Step 2: Train N-gram POS tagging model (Bigram)
def train_ngram_pos_tagger(data, n=2):
    ngram_counts = defaultdict(lambda: defaultdict(int))
    pos_counts = defaultdict(int)
    
    for sentence, pos_tags in data:
        pos_tags = ['<START>'] * (n-1) + pos_tags + ['<END>']  # Padding for N-grams
        for i in range(len(pos_tags) - n + 1):
            ngram = tuple(pos_tags[i:i+n-1])
            next_pos = pos_tags[i+n-1]
            ngram_counts[ngram][next_pos] += 1
            pos_counts[next_pos] += 1
    
    # Convert counts to probabilities
    ngram_probs = {ngram: {pos: count / sum(next_pos_dict.values())
                           for pos, count in next_pos_dict.items()}
                   for ngram, next_pos_dict in ngram_counts.items()}
    
    return ngram_probs, pos_counts

# Step 3: Predict POS tags using the N-gram model
def predict_pos_ngram(sentence, ngram_probs, pos_counts, n=2):
    pos_tags = ['<START>'] * (n-1)
    for word in sentence:
        ngram = tuple(pos_tags[-(n-1):])  # Use last (n-1) POS tags
        if ngram in ngram_probs:
            predicted_pos = max(ngram_probs[ngram], key=ngram_probs[ngram].get)
        else:
            predicted_pos = max(pos_counts, key=pos_counts.get)  # Fallback to most common POS
        pos_tags.append(predicted_pos)
    return pos_tags[n-1:]  # Remove padding

# Load your dataset
train_data_path = "UD_Chinese-GSDSimp/zh_gsdsimp-ud-train.conllu"
train_data = read_conllu_data(train_data_path)

# Train the N-gram model (bigram here)
ngram_probs, pos_counts = train_ngram_pos_tagger(train_data, n=2)

# Test N-gram POS Tagging on a sample sentence
sample_sentence = ["我", "爱", "学习"]  # Example Chinese sentence
predicted_pos_tags = predict_pos_ngram(sample_sentence, ngram_probs, pos_counts, n=2)

print("Sample sentence:", sample_sentence)
print("Predicted POS tags:", predicted_pos_tags)

Sample sentence: ['我', '爱', '学习']
Predicted POS tags: ['PROPN', 'PART', 'NOUN']


### SENTIMENT POLARITY

In [126]:
!pip install SnowNLP

     ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
     --------------------------------------- 0.0/37.6 MB 660.6 kB/s eta 0:00:57
     --------------------------------------- 0.1/37.6 MB 751.6 kB/s eta 0:00:50
     ---------------------------------------- 0.2/37.6 MB 1.2 MB/s eta 0:00:30
     ---------------------------------------- 0.3/37.6 MB 1.5 MB/s eta 0:00:26
     ---------------------------------------- 0.4/37.6 MB 1.5 MB/s eta 0:00:25
     ---------------------------------------- 0.4/37.6 MB 1.5 MB/s eta 0:00:25
      --------------------------------------- 0.5/37.6 MB 1.4 MB/s eta 0:00:26
      --------------------------------------- 0.5/37.6 MB 1.4 MB/s eta 0:00:26
      --------------------------------------- 0.6/37.6 MB 1.3 MB/s eta 0:00:28
      --------------------------------------- 0.7/37.6 MB 1.4 MB/s eta 0:00:27
      --------------------------------------- 0.7/37.6 MB 1.4 MB

In [128]:
from snownlp import SnowNLP

# Step 2: Perform sentiment polarity analysis using SnowNLP
def analyze_sentiment_snownlp(sentences):
    for sentence in sentences:
        s = SnowNLP(sentence)
        sentiment_polarity = s.sentiments  # Sentiment score ranges from 0 (negative) to 1 (positive)
        print(f"Sentence: {sentence}")
        print(f"Sentiment polarity: {sentiment_polarity:.4f}\n")

# Analyze sentiment using SnowNLP
analyze_sentiment_snownlp(sentences[:5])


Sentence: 看似 简单 ， 只 是 二 选 一 做 决择 ， 但 其实 他们 代表 的 是 你 周遭 的 亲朋 好友 ， 试 着 给 你 不同 的 意见 ， 但 追根究底 ， 最后 决定 的 还是 自己 。
Sentiment polarity: 0.9988

Sentence: 其 便当 都是 买来 的 ， 就算 加热 也是 由 妈妈 负责 （ 后来 揭晓 其实 是 避免 带来 厄运 ） ， 父亲 则 在 电视 台 上班 。
Sentiment polarity: 0.9112

Sentence: 这 次 游行 最大 的 特色 ， 在 于 越来越 多 年轻 人 上街 游行 ， 而且 当中 不乏 行动 激烈 的 躁 少年 。
Sentiment polarity: 0.9954

Sentence: 怀孕 期 为 421 至 457 日 。
Sentiment polarity: 0.7628

Sentence: 婷婷 向 昏迷 中 的 婆婆 诉说 ， 为 什么 生活 会 与 她 想像 的 不 一样 。
Sentiment polarity: 0.9977



### OOV

In [24]:
from collections import defaultdict

class HMMTagger:
    def __init__(self, training_data):
        self.training_data = training_data
        self.vocab = set()  # Vocabulary from training
        self.tag_count = defaultdict(int)  # Count of each POS tag
        self.word_tag_count = defaultdict(lambda: defaultdict(int))  # Count of word given a tag
        self.tag_transition_count = defaultdict(lambda: defaultdict(int))  # Count of tag transitions
        self.smoothing_value = 1e-6  # Small smoothing value for OOV handling

    def train(self):
        prev_tag = "<START>"
        for sentence in self.training_data:
            for word, tag in sentence:
                self.vocab.add(word)
                self.tag_count[tag] += 1
                self.word_tag_count[tag][word] += 1
                self.tag_transition_count[prev_tag][tag] += 1
                prev_tag = tag
            self.tag_transition_count[prev_tag]["<END>"] += 1

    def emission_probability(self, word, tag):
        """
        Calculates the emission probability P(word|tag).
        For OOV words, assign a small smoothing probability.
        """
        if word in self.vocab:
            return (self.word_tag_count[tag][word] + self.smoothing_value) / (self.tag_count[tag] + self.smoothing_value)
        else:
            # Handle OOV: Assign small probability
            return self.smoothing_value / (self.tag_count[tag] + self.smoothing_value)

    def transition_probability(self, prev_tag, current_tag):
        """
        Calculates the transition probability P(tag|prev_tag).
        """
        return (self.tag_transition_count[prev_tag][current_tag] + self.smoothing_value) / \
               (sum(self.tag_transition_count[prev_tag].values()) + self.smoothing_value)

    def viterbi(self, sentence):
        """
        Viterbi algorithm to find the most likely sequence of tags for a sentence.
        """
        V = [{}]
        path = {}

        # Initialize base cases (start probabilities)
        for tag in self.tag_count:
            V[0][tag] = self.transition_probability("<START>", tag) * self.emission_probability(sentence[0], tag)
            path[tag] = [tag]

        # Run Viterbi for each word in the sentence
        for t in range(1, len(sentence)):
            V.append({})
            newpath = {}

            for tag in self.tag_count:
                # For each tag, find the highest probability from previous tags
                (prob, best_prev_tag) = max(
                    (V[t - 1][prev_tag] * self.transition_probability(prev_tag, tag) *
                     self.emission_probability(sentence[t], tag), prev_tag)
                    for prev_tag in self.tag_count
                )

                V[t][tag] = prob
                newpath[tag] = path[best_prev_tag] + [tag]

            # Update path
            path = newpath

        # Termination: Find the best final tag sequence
        (prob, best_tag) = max((V[len(sentence) - 1][tag], tag) for tag in self.tag_count)

        return (prob, path[best_tag])

# Sample training data: list of sentences where each sentence is a list of (word, tag) pairs
training_data = [
    [('I', 'PRON'), ('am', 'AUX'), ('happy', 'ADJ')],
    [('She', 'PRON'), ('is', 'AUX'), ('here', 'ADV')],
    # Add more training data
]

# Initialize the tagger and train
hmm_tagger = HMMTagger(training_data)
hmm_tagger.train()

# Sample test sentence with an OOV word
test_sentence = ['他', '感到', '高兴']  # 'He feels happy'

# Run Viterbi decoding
prob, best_tag_sequence = hmm_tagger.viterbi(test_sentence)
print(f"Best tag sequence: {best_tag_sequence}")


Best tag sequence: ['PRON', 'AUX', 'ADV']


### NER - NAMED ENTITY RECOGNITION

In [20]:
# Step 1: Load and preprocess the CoNLL formatted data for NER
def read_conllu_data_ner(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        sentence = []
        ner_tags = []
        for line in f:
            if line.strip() == "":
                if sentence and ner_tags:
                    data.append((sentence, ner_tags))
                    sentence = []
                    ner_tags = []
            else:
                parts = line.strip().split("\t")
                if len(parts) > 1:
                    sentence.append(parts[1])  # Word
                    ner_tags.append(parts[5])  # NER tag
    return data

# Step 2: HMM Training - Calculate transition and emission probabilities for NER
def train_hmm_ner(data):
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    ner_counts = defaultdict(int)
    
    for sentence, ner_tags in data:
        prev_ner = "<START>"
        for i in range(len(sentence)):
            word, ner = sentence[i], ner_tags[i]
            transition_counts[prev_ner][ner] += 1
            emission_counts[ner][word] += 1
            ner_counts[ner] += 1
            prev_ner = ner
        # Mark the end of the sentence
        transition_counts[prev_ner]["<END>"] += 1
    
    # Convert counts to probabilities
    transition_probs = {prev_ner: {ner: count / sum(next_ner_dict.values())
                                   for ner, count in next_ner_dict.items()}
                        for prev_ner, next_ner_dict in transition_counts.items()}
    
    emission_probs = {ner: {word: count / sum(word_dict.values())
                            for word, count in word_dict.items()}
                      for ner, word_dict in emission_counts.items()}
    
    return transition_probs, emission_probs, ner_counts

# Step 3: Viterbi Algorithm for NER tagging
def viterbi_ner(sentence, transition_probs, emission_probs, ner_counts):
    ner_tags = list(ner_counts.keys())
    
    # Initialize Viterbi matrix and backpointer matrix
    viterbi_matrix = np.zeros((len(ner_tags), len(sentence)))
    backpointer = np.zeros((len(ner_tags), len(sentence)), dtype=int)
    
    # Initialization step
    for i, ner in enumerate(ner_tags):
        emission_prob = emission_probs[ner].get(sentence[0], 1e-6)  # Smoothing for unseen words
        viterbi_matrix[i, 0] = transition_probs["<START>"].get(ner, 1e-6) * emission_prob
    
    # Recursion step
    for t in range(1, len(sentence)):
        for i, ner in enumerate(ner_tags):
            max_prob = -1
            max_state = 0
            for j, prev_ner in enumerate(ner_tags):
                prob = (viterbi_matrix[j, t-1] * 
                        transition_probs[prev_ner].get(ner, 1e-6) * 
                        emission_probs[ner].get(sentence[t], 1e-6))
                if prob > max_prob:
                    max_prob = prob
                    max_state = j
            viterbi_matrix[i, t] = max_prob
            backpointer[i, t] = max_state
    
    # Termination step
    best_last_state = np.argmax(viterbi_matrix[:, len(sentence)-1])
    
    # Backtrack to find the best path
    best_path = [best_last_state]
    for t in range(len(sentence)-1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, best_last_state)
    
    # Convert state indices back to NER tags
    best_ner_sequence = [ner_tags[state] for state in best_path]
    return best_ner_sequence

# Step 4: Calculate accuracy
def calculate_accuracy_ner(predicted_tags, true_tags):
    correct = sum(p == t for p, t in zip(predicted_tags, true_tags))
    return correct / len(true_tags) if true_tags else 0.0

# Load your dataset (you will need a NER dataset similar to CoNLL 2003)
train_data_path = "UD_Chinese-GSDSimp/zh_gsdsimp-ud-train.conllu"
test_data_path = "UD_Chinese-GSDSimp/zh_gsdsimp-ud-test.conllu"

train_data = read_conllu_data_ner(train_data_path)
test_data = read_conllu_data_ner(test_data_path)

# Train the HMM model for NER
transition_probs, emission_probs, ner_counts = train_hmm_ner(train_data)

# Test the HMM model for NER and calculate accuracy
all_predicted_tags = []
all_true_tags = []

for sentence, true_ner_tags in test_data:
    predicted_tags = viterbi_ner(sentence, transition_probs, emission_probs, ner_counts)
    all_predicted_tags.extend(predicted_tags)
    all_true_tags.extend(true_ner_tags)

accuracy = calculate_accuracy_ner(all_predicted_tags, all_true_tags)

# Output accuracy
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9665


In [32]:
from collections import defaultdict
import numpy as np

# Step 1: Load and preprocess the CoNLL formatted data for NER (Name, Place, Animal, Thing)
def read_conllu_data_ner(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        sentence = []
        ner_tags = []
        for line in f:
            if line.strip() == "":
                if sentence and ner_tags:
                    data.append((sentence, ner_tags))
                    sentence = []
                    ner_tags = []
            else:
                parts = line.strip().split("\t")
                if len(parts) > 1:
                    sentence.append(parts[1])  # Word
                    ner_tags.append(parts[5])  # NER tag (should include B-NAME, B-PLACE, etc.)
    return data

# Step 2: HMM Training - Calculate transition and emission probabilities for NER
def train_hmm_ner(data):
    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    ner_counts = defaultdict(int)
    
    for sentence, ner_tags in data:
        prev_ner = "<START>"
        for i in range(len(sentence)):
            word, ner = sentence[i], ner_tags[i]
            transition_counts[prev_ner][ner] += 1
            emission_counts[ner][word] += 1
            ner_counts[ner] += 1
            prev_ner = ner
        # Mark the end of the sentence
        transition_counts[prev_ner]["<END>"] += 1
    
    # Convert counts to probabilities
    transition_probs = {prev_ner: {ner: count / sum(next_ner_dict.values())
                                   for ner, count in next_ner_dict.items()}
                        for prev_ner, next_ner_dict in transition_counts.items()}
    
    emission_probs = {ner: {word: count / sum(word_dict.values())
                            for word, count in word_dict.items()}
                      for ner, word_dict in emission_counts.items()}
    
    return transition_probs, emission_probs, ner_counts

# Step 3: Viterbi Algorithm for NER tagging (Name, Place, Animal, Thing)
def viterbi_ner(sentence, transition_probs, emission_probs, ner_counts):
    ner_tags = list(ner_counts.keys())
    
    # Initialize Viterbi matrix and backpointer matrix
    viterbi_matrix = np.zeros((len(ner_tags), len(sentence)))
    backpointer = np.zeros((len(ner_tags), len(sentence)), dtype=int)
    
    # Initialization step
    for i, ner in enumerate(ner_tags):
        emission_prob = emission_probs[ner].get(sentence[0], 1e-6)  # Smoothing for unseen words
        viterbi_matrix[i, 0] = transition_probs["<START>"].get(ner, 1e-6) * emission_prob
    
    # Recursion step
    for t in range(1, len(sentence)):
        for i, ner in enumerate(ner_tags):
            max_prob = -1
            max_state = 0
            for j, prev_ner in enumerate(ner_tags):
                prob = (viterbi_matrix[j, t-1] * 
                        transition_probs[prev_ner].get(ner, 1e-6) * 
                        emission_probs[ner].get(sentence[t], 1e-6))
                if prob > max_prob:
                    max_prob = prob
                    max_state = j
            viterbi_matrix[i, t] = max_prob
            backpointer[i, t] = max_state
    
    # Termination step
    best_last_state = np.argmax(viterbi_matrix[:, len(sentence)-1])
    
    # Backtrack to find the best path
    best_path = [best_last_state]
    for t in range(len(sentence)-1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, best_last_state)
    
    # Convert state indices back to NER tags
    best_ner_sequence = [ner_tags[state] for state in best_path]
    return best_ner_sequence

# Step 4: Calculate accuracy
def calculate_accuracy_ner(predicted_tags, true_tags):
    correct = sum(p == t for p, t in zip(predicted_tags, true_tags))
    return correct / len(true_tags) if true_tags else 0.0

# Load your dataset (you will need a NER dataset with Name, Place, Animal, Thing tags)

train_data = read_conllu_data_ner(train_data_path)
test_data = read_conllu_data_ner(test_data_path)

# Train the HMM model for NER
transition_probs, emission_probs, ner_counts = train_hmm_ner(train_data)

# Test the HMM model for NER and calculate accuracy
all_predicted_tags = []
all_true_tags = []

for sentence, true_ner_tags in test_data:
    predicted_tags = viterbi_ner(sentence, transition_probs, emission_probs, ner_counts)
    
    # Print the sentence along with the predicted and true NER tags
    print("\nSentence:", " ".join(sentence))
    for word, pred_tag, true_tag in zip(sentence, predicted_tags, true_ner_tags):
        print(f"Word: {word}, Predicted NER: {pred_tag}, True NER: {true_tag}")
    
    all_predicted_tags.extend(predicted_tags)
    all_true_tags.extend(true_ner_tags)

# Calculate overall accuracy
accuracy = calculate_accuracy_ner(all_predicted_tags, all_true_tags)

# Output accuracy
print(f"\nAccuracy: {accuracy:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_train_data.conllu'